In [1]:
import warnings
warnings.filterwarnings('ignore')

# <span style="color:Maroon">Case Study: Usage of Data Science in Entertainment Industry.

#### <span style="color:Green">Select optimal advetisement to display in break time for a TV Show, based on what was recently shown in the TV

## <span style="color:Maroon">Naive Classifier (Based on Majority Class)

<span style="color:Green">A simple implementation of Majority Class Classifier to show, how accuracy in itself is not a good metric of performance. The model just picks the marjority class, and always predict, it as the output. In biased dataset (One class is more prevalent over others), this model will do really good in terms of accuracy. 

<span style="color:Green">__NEVER USE THIS AS FINAL MODEL. THIS IS JUST FOR ILLUSTRATION PURPOSE__

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import os as os
np.random.seed(0)
pd.option_context('display.max_rows', None, 'display.max_columns', None)

In [3]:
# Setting the working directory to the folder
direc = os.getcwd()
os.chdir("..//Data//")

In [4]:
# Read the data in pandas dataframe
data = pd.read_csv("Dataset.csv")
data = data.fillna(0)

In [5]:
# Dependent variable (is given name of target)
target = 'h_food_str'
# Other dependent variables (These should be dropped from our dataset)
dv_list = ['h_alcohol_str', 'h_hot_drink_str', 'h_phone_str']
# drop the dv_list from the data (as In future, these labels won't be available to us (dv_list).
# Hence, for our modeling purpose, these variables are being dropped from the data.)
data = data.drop(dv_list, axis=1)

In [6]:
# One hot encoding of 'show_name' variable
df1 = pd.get_dummies(data['show_name'], prefix = 'show_name')
data = data.join(df1)
data.head()

,show_name,time_offset,r_abies,r_abyssinian,r_accessories,r_accipiter,r_acorn,r_adapter,r_adorable,r_adventure,...,r_zoo,h_food_str,show_name_fresh_meat,show_name_friday_night_dinner,show_name_hollyoaks,show_name_made_in_chelsea,show_name_made_in_chelsea_la,show_name_my_mad_fat_diary,show_name_peep_show,show_name_the_inbetweeners
0,fresh_meat,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,1,0,0,0,0,0,0,0
1,fresh_meat,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,1,0,0,0,0,0,0,0
2,fresh_meat,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,1,0,0,0,0,0,0,0
3,fresh_meat,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,1,0,0,0,0,0,0,0
4,fresh_meat,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,1,0,0,0,0,0,0,0


### <span style="color:Maroon">Train-Test Split
<span style="color:Green">Divide the data in 70:30 ratio for training the model and validating the model

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# Split the data in train and test
X_train, X_test, y_train, y_test = train_test_split(data.drop([target],axis=1), data[target], test_size=0.30, 
                                                    random_state=12345)

In [9]:
y_train = y_train.ravel()
y_test = y_test.ravel()

In [10]:
# Check if the random samples have similar mean
print (f"Train Dependent variable: {np.round(np.mean(y_train)*100,2)}")
print (f"Test Dependent variable: {np.round(np.mean(y_test)*100,2)}")

Train Dependent variable: 8.83
Test Dependent variable: 8.85


<span style="color:Blue">__Comments:__ The means of dependent variable are very similar in test and train dataset

### <span style="color:Maroon">Part 1: Naive Classifier:
##### The classifier just predicts the majority class as the output.

In [11]:
# import base model libraries from sklearn
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.metrics import classification_report

<span style="color:Green">Define a clas MajorityClassifier to predicit the output as class that is most common

In [12]:
# Define a class to predict it to resue in case of future
class MajorityClassifier(BaseEstimator, ClassifierMixin):
    def __self__(self):
        pass

    def fit(self, X, y):
        c = Counter(y)
        self.mode_ = c.most_common(1)[0][0]
        self.fraction_ = np.array(list(c.values()))/X.shape[0]
        return self

    def predict(self, X):
        return self.mode_ * np.ones(X.shape[0])
    
    
    def predict_proba(self, X):
        return np.dot(np.ones((X.shape[0],1)), self.fraction_.reshape(1,-1))

In [13]:
# Fitting the majority class model
mc = MajorityClassifier()
mc.fit(X_train,y_train)

MajorityClassifier()

In [14]:
# Prediciting the class for training dataset
y_train_class_pred = mc.predict(X_train)

In [15]:
# Lets look at the performance of the naive model
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
print("Performance Metrics for Training Sample")
print(classification_report(y_train, y_train_class_pred))

tmp = pd.DataFrame(y_train, columns=["Actual"])
tmp["Predicted"] = y_train_class_pred
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("\nConfusion Matrix\n")
print(pd.crosstab(tmp["Actual"],tmp["Predicted"]))

+++++++++++++++++++++++++++++++++++++++++++++++++++++

Performance Metrics for Training Sample
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     10573
           1       0.00      0.00      0.00      1024

    accuracy                           0.91     11597
   macro avg       0.46      0.50      0.48     11597
weighted avg       0.83      0.91      0.87     11597

+++++++++++++++++++++++++++++++++++++++++++++++++++++

Confusion Matrix

Predicted    0.0
Actual          
0          10573
1           1024


<span style="color:Blue">__Comments:__ Though the model has an accuracy of 91%, the precision, recall and f1-score are all 0's (for class == 1). This is not a good model (although if we look only at accuracy, it might look a very good model)

In [16]:
y_test_class_pred = mc.predict(X_test)

# Peroformance for test sample:
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
print("Performance Metrics for Test Sample")
print(classification_report(y_test, y_test_class_pred))

tmp = pd.DataFrame(y_test, columns=["Actual"])
tmp["Predicted"] = y_test_class_pred
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("\nConfusion Matrix\n")
print(pd.crosstab(tmp["Actual"],tmp["Predicted"]))

+++++++++++++++++++++++++++++++++++++++++++++++++++++

Performance Metrics for Test Sample
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4531
           1       0.00      0.00      0.00       440

    accuracy                           0.91      4971
   macro avg       0.46      0.50      0.48      4971
weighted avg       0.83      0.91      0.87      4971

+++++++++++++++++++++++++++++++++++++++++++++++++++++

Confusion Matrix

Predicted   0.0
Actual         
0          4531
1           440


<span style="color:Blue">__Comments__ Same as training sample, the model has a really good accuracy, but recall, precision and f1-score are all 0's (for class == 1). 